In [ ]:
import sys
import asyncio
import time
import os

import numpy as np

from lsst.ts import salobj

from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS
from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import astropy.units as u

In [ ]:
# Set Cerro Pachon location
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

In [ ]:
import logging
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [ ]:
#Start classes
domain = salobj.Domain()
await asyncio.sleep(10) # This can be removed in the future...
atcs = ATCS(domain)
latiss = LATISS(domain)
await asyncio.gather(atcs.start_task, latiss.start_task)

In [ ]:
# enable components
await atcs.enable({"atdome": "current", "ataos": "current", "athexapod": "current"})
await latiss.enable()

In [ ]:
# ATPneumatics didn't come up
tmp=await atcs.rem.atpneumatics.cmd_start.set_start()
print(tmp)

In [ ]:
await salobj.set_summary_state(atcs.rem.atptg, salobj.State.ENABLED)

In [ ]:
await salobj.set_summary_state(atcs.rem.atmcs, salobj.State.ENABLED)

In [ ]:
# ATPneumatics didn't come up
tmp=await atcs.rem.atdome.cmd_start.set_start()
print(tmp)

In [ ]:
# ATPneumatics didn't come up
tmp=await atcs.rem.atdometrajectory.cmd_start.set_start()
print(tmp)

In [ ]:
await salobj.set_summary_state(atcs.rem.atdome, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(atcs.rem.atdometrajectory, salobj.State.ENABLED)

In [ ]:
# ATArchiver didn't come up
tmp=await atcs.rem.atptg.cmd_start.set_start()
print(tmp)

In [ ]:
await salobj.set_summary_state(latiss.rem.atarchiver, salobj.State.ENABLED)

In [ ]:
await latiss.rem.atarchiver.cmd_enable.set_start(timeout=10)

In [ ]:
# These 8 cells were run earlier to install the labJack in a box

In [ ]:
# Lowering to check box clearance
start_az = 0.0
start_el = 75.0
start_rot = 0.0
await atcs.point_azel(start_az, start_el, rot_tel=start_rot)

In [ ]:
await salobj.set_summary_state(atcs.rem.atmcs, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(atcs.rem.atptg, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(atcs.rem.atmcs, salobj.State.ENABLED)

In [ ]:
await salobj.set_summary_state(atcs.rem.atptg, salobj.State.ENABLED)

In [ ]:
# Failed.  Bring back up
start_az = 0.0
start_el = 80.0
start_rot = 0.0
await atcs.point_azel(start_az, start_el, rot_tel=start_rot)

In [ ]:
# Testing new location
start_az = 0.0
start_el = 20.0
start_rot = 0.0
await atcs.point_azel(start_az, start_el, rot_tel=start_rot)

In [ ]:
# Looks OK.  Bring back up
start_az = 0.0
start_el = 80.0
start_rot = 0.0
await atcs.point_azel(start_az, start_el, rot_tel=start_rot)

In [ ]:
# Running more tests

In [ ]:
await latiss.take_bias(1)

In [ ]:
# Take 50 biases seq # 5-54
# Added wait to stop killing the recent images
for i in range(50):
    await asyncio.sleep(2.0)
    await latiss.take_bias(1)

In [ ]:
# Re-aligning flat illuminator
await atcs.prepare_for_flatfield()

In [ ]:
await atcs.prepare_for_flatfield()

In [ ]:
await atcs.stop_tracking()

In [ ]:
await atcs.shutdown()

In [ ]:
await latiss.standby()